# Identify similar question pair that have same meaning or intent.

#### Project Planning

- **Data Collection** - Using Kaggle API for calling dataset
- **Data Cleaning** - Preprocesisng removing question marks, hyphens and dull stops
- **Exploratory Data Analysis** - *Not required*
- **Model Building** - Using state of the art NLP Algo - **BERT**, Jaccard, TF-IDF, Doc2vec, USE, **cosine-similarity**
- **Putting Model in Production** - *not required*
- **Reviewing and Documenting Work** - Final PPT and mid eval reports

Links -
1. https://towardsdatascience.com/the-best-document-similarity-algorithm-in-2020-a-beginners-guide-a01b9ef8cf05
2. https://www.kaggle.com/c/quora-question-pairs/


In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"vatsalbajaj","key":"964b67bdf845b0831d8c7b0066997590"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c quora-question-pairs

 57% 12.0M/21.2M [00:00<00:00, 121MB/s]
100% 21.2M/21.2M [00:00<00:00, 104MB/s]
  0% 0.00/4.95M [00:00<?, ?B/s]
100% 4.95M/4.95M [00:00<00:00, 78.4MB/s]
 90% 103M/114M [00:00<00:00, 114MB/s] 
100% 114M/114M [00:00<00:00, 126MB/s]
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
pwd

'/content'

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# from subprocess import check_output
!ls     
# print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_data  sample_submission.csv.zip	test.csv.zip  train.csv.zip


In [ ]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

### Data Cleaning and Preprocessing
1. Remove not required feature columns
2. remove punctuation and full stops and if required ?'s (a hypothesis)
3. Choose the tokenizing method - like word tokenzier

In [ ]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
test_df.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [ ]:
print(train_df['question2'][3])

Find the remainder when [math]23^{24}[/math] is divided by 24,23?


In [ ]:
len(train_df)+len(test_df)

2750086

In [ ]:
train_df = train_df.drop(labels=["id","qid1","qid2","is_duplicate"],axis=1)
test_df= test_df.drop(["test_id"],axis=1)

In [ ]:
df = pd.concat([train_df,test_df])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2750086 entries, 0 to 2345795
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   question1  object
 1   question2  object
dtypes: object(2)
memory usage: 62.9+ MB


In [ ]:
df[:50]

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan..."
6,Should I buy tiago?,What keeps childern active and far from phone ...
7,How can I be a good geologist?,What should I do to be a great geologist?
8,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?"
9,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?


### Create model, BERT

### Testing -